# Utility to clean the transport and thermodynamic properties of a kinetic mechanism

The aim of this utility is to write a transport and thermodynamic CHEMKIN file including only the species which are part of the kinetic mechanism, i.e. removing all the specis which are not relevant.
The user has to provide the path to the original transport and thermodynamic CHEMKIN file (which are assumed to be properly written) and an additional ASCII file containing the list of all the species included in the final kinetic mechanism. The utility will automatically produce new "clean" transport and thermodynamic CHEMKIN files. The user can also specify some header lines to be added on the top of the files, for example reporting the details of the paper to which the mechanism is attached.
In case of error message about the decodification of input files, we suggest to change the econding parameter from 'latin-1' to 'utf-8'

In [1]:
# Input data
encoding_format = 'latin-1'
list_of_species_file_name = 'listofspecies'
transport_file_name = 'CRECK_2003_C1_C3_HT_NOX.TRC'
thermo_file_name = 'CRECK_2003_C1_C3_HT_NOX.CKT'
header_lines = \
'! ------------------------------------------------------------------------------- ! \n' + \
'!                                CRECK Modeling Lab                               ! \n' + \
'!                       http://creckmodeling.chem.polimi.it                       ! \n' + \
'!                                                                                 ! \n' + \
'!                     Detailed Kinetic Mechanism for n-heptane                    ! \n' + \
'!                                                                                 ! \n' + \
'! Pelucchi M., Faravelli T., Frassoldati A., Article title, Journal, Year         ! \n' + \
'! DOI: https://doi.org/XX.YYYY/xxx.yyy                                            ! \n' + \
'! ------------------------------------------------------------------------------- ! \n' 

Reading the list of species

In [2]:
# Read list of species
listofspecies_file = open(list_of_species_file_name, 'r', encoding=encoding_format)
listofspecies = (listofspecies_file.read()).split()
ns = len(listofspecies)

Cleaning the transport file

In [3]:
tran_file = open(transport_file_name, 'r', encoding=encoding_format)
tran_data = tran_file.readlines()
tran_lines = len(tran_data)

data = []
found = [False]*ns
for i in range(ns):
    species = listofspecies[i]
    for j in range(tran_lines):
        if (found[i] == False):
            words = tran_data[j].split()
            if (len(words) != 0):
                if (words[0] == species):
                    found[i] = True
                    single_data = { 'name': species, 'coeffs': words[1:7] }
                    data.append(single_data)
                    
additional_comments = \
'! Molecular parameter for each species' + '\n' \
'! 1. An  index  indicating  whether the molecule has a monatomic, linear' + '\n' \
'!    or nonlinear geometrical configuration. If the index is 0, the' + '\n' \
'!    molecule is a single atom. If the index is 1 the molecule is linear,' + '\n' \
'!    and if it is 2, the molecule is nonlinear.' + '\n' \
'! 2. The Lennard-Jones potential well depth eps/kB  in Kelvins' + '\n' \
'! 3. The Lennard-Jones collision diameter sigma in Angstroms ' + '\n' \
'! 4. The dipole moment mu in Debye. Note: a Debye is 1e-18 cm3/2 erg1/2 ' + '\n' \
'! 5. The polarizability alpha in cubic Angstroms.' + '\n' \
'! 6. The rotational relaxation collision number Zrot at 298K.' + '\n' 

f = open(transport_file_name+'.clean', 'w')
f.write(header_lines + '\n')
f.write(additional_comments + '\n')
f.write('%-30s %-12s %-12s %-12s %-12s %-12s %-12s\n' % ('!', 'config.', 'eps/kb(K)', 'sigma(A)', \
                                                         'moment(Db)', 'polar.(A^3)', 'Zrot@298K' ) )
for i in range(ns):
    f.write('%-30s %-12s %-12s %-12s %-12s %-12s %-12s\n' % (data[i]['name'], data[i]['coeffs'][0], \
                                                             data[i]['coeffs'][1],data[i]['coeffs'][2], \
                                                             data[i]['coeffs'][3],data[i]['coeffs'][4],\
                                                             data[i]['coeffs'][5]) )

f.close()

Cleaning the thermodynamic file

In [4]:
thermo_file = open(thermo_file_name, 'r', encoding=encoding_format)
thermo_data = thermo_file.readlines()
thermo_lines = len(thermo_data)

data = []
found = [False]*ns
for i in range(ns):
    species = listofspecies[i]
    for j in range(thermo_lines):
        if (found[i] == False):
            words = thermo_data[j].split()
            if (len(words) != 0):
                if (words[0] == species):
                    found[i] = True
                    if (thermo_data[j][80] == '&'):
                        nlines = 5;
                        first_line = thermo_data[j][0:81]
                        second_line = thermo_data[j+1].strip()
                        last_line = thermo_data[j+4][0:80]
                    else:
                        nlines = 4
                        first_line = thermo_data[j][0:80]
                        second_line = thermo_data[j+1][0:80]
                        last_line = ''
                    
                    single_data = { 'name': species, 'nlines': nlines, \
                                    'line1': first_line+'\n', \
                                    'line2': second_line+'\n', \
                                    'line3': thermo_data[j+2][0:80]+'\n', \
                                    'line4': thermo_data[j+3][0:80]+'\n', \
                                    'line5': last_line+'\n' }
                    
                    data.append(single_data)

f = open(thermo_file_name + '.clean', 'w')
f.write(header_lines + '\n')
f.write('THERMO ALL' + '\n')
f.write('270.   1000.   3500. ' + '\n')

for i in range(ns):
    f.write(data[i]['line1'])
    f.write(data[i]['line2'])
    f.write(data[i]['line3'])
    f.write(data[i]['line4'])
    if (data[i]['nlines'] == 5):
        f.write(data[i]['line5'])
        
f.write('END' + '\n')
f.write('\n')
f.close()